РК№2 Вариант 13 Линейная/логистическая регрессия

Случайный лес


Садиров Ф.Э. ИУ5-61Б

In [14]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier

Загрузка данных

In [4]:
# Загрузим данные
data = pd.read_csv('marvel.csv')

# Удаление бесполезного столбца GSM
data.drop(columns=['GSM'], inplace=True)

# Посмотрим на первые несколько строк данных и проверим наличие пропусков
data.head()


,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Living Characters,2258.0,Nov-50,1950.0


In [ ]:
data.dtypes

page_id               int64
name                 object
urlslug              object
ID                   object
ALIGN                object
EYE                  object
HAIR                 object
SEX                  object
ALIVE                object
APPEARANCES         float64
FIRST APPEARANCE     object
Year                float64
dtype: object

In [ ]:
data.isnull().sum()

page_id                0
name                   0
urlslug                0
ID                  3770
ALIGN               2812
EYE                 9767
HAIR                4264
SEX                  854
ALIVE                  3
APPEARANCES         1096
FIRST APPEARANCE     815
Year                 815
dtype: int64

# Обработка пропущенных значений

In [5]:
data['APPEARANCES'].fillna(data['APPEARANCES'].median(), inplace=True)
data['Year'].fillna(data['Year'].median(), inplace=True)

for column in ['name', 'urlslug', 'ID', 'ALIGN', 'EYE', 'HAIR', 'SEX', 'ALIVE', 'FIRST APPEARANCE']:
    data[column].fillna(data[column].mode()[0], inplace=True)


# Кодирование категориальных признаков

Используем LabelEncoder для категориальных признаков. Align будет нашим целевым признаком.

In [7]:
le = LabelEncoder()

for column in ['ID', 'ALIGN', 'EYE', 'HAIR', 'SEX', 'ALIVE', 'FIRST APPEARANCE']:
    data[column] = le.fit_transform(data[column])

# Построение моделей

In [9]:
X = data.drop(columns=['ALIGN', 'page_id', 'name', 'urlslug'])
y = data['ALIGN']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Логистическая регрессия

In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создаем модель логистической регрессии
logistic_model = LogisticRegression(max_iter=500, solver='liblinear')
logistic_model.fit(X_train_scaled, y_train)

# Предсказания
y_pred_logistic = logistic_model.predict(X_test_scaled)

# Оценка модели
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
f1_logistic = f1_score(y_test, y_pred_logistic, average='weighted')

print(f'Accuracy (Logistic Regression): {accuracy_logistic}')
print(f'F1 Score (Logistic Regression): {f1_logistic}')

Accuracy (Logistic Regression): 0.6190476190476191
F1 Score (Logistic Regression): 0.5349284037363744


Случайный лес

In [19]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

# Оценка модели
accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

print(f'Accuracy (Random Forest): {accuracy_rf}')
print(f'F1 Score (Random Forest): {f1_rf}')

Accuracy (Random Forest): 0.6248473748473748
F1 Score (Random Forest): 0.5965120527214235


# Оценка моделей

Мы использовали две метрики для оценки моделей:

* Accuracy (точность) показывает, какая доля предсказаний модели оказалась верной.
* F1 Score (взвешенная) учитывает как точность (precision), так и полноту (recall) модели, что особенно важно при дисбалансе классов.

# Выводы
* Логистическая регрессия: показала достаточно хорошую точность и F1 Score, но может страдать при нелинейных зависимостях в данных.
* Случайный лес: показал более высокие значения точности и F1 Score по сравнению с логистической регрессией, что говорит о его способности лучше справляться с нелинейностями и сложными взаимодействиями признаков.

По результатам видно, что случайный лес показывает лучшие результаты для данной задачи классификации, что делает его предпочтительным выбором.